In [2]:
%maven org.nd4j:nd4j-cuda-9.0:1.0.0-beta
%maven org.deeplearning4j:deeplearning4j-cuda-9.0:1.0.0-beta
%maven org.datavec:datavec-api:1.0.0-beta
%maven org.knowm.xchart:xchart:3.5.2
    
%maven org.scala-lang:scala-library:2.13.0-M5-6e0cba7

In [11]:
import org.deeplearning4j.datasets.iterator.impl.MnistDataSetIterator;
import org.deeplearning4j.nn.api.OptimizationAlgorithm;
import org.deeplearning4j.nn.conf.MultiLayerConfiguration;
import org.deeplearning4j.nn.conf.NeuralNetConfiguration;
import org.deeplearning4j.nn.conf.Updater;
import org.deeplearning4j.nn.conf.layers.DenseLayer;
import org.deeplearning4j.nn.conf.layers.OutputLayer;
import org.deeplearning4j.nn.multilayer.MultiLayerNetwork;
import org.deeplearning4j.nn.weights.WeightInit;
import org.deeplearning4j.optimize.solvers.StochasticGradientDescent;
import org.nd4j.linalg.activations.Activation;
import org.nd4j.linalg.dataset.api.iterator.DataSetIterator;
import org.nd4j.linalg.lossfunctions.LossFunctions;
import org.nd4j.linalg.learning.config.Sgd;
import java.util.ArrayList;

import org.nd4j.linalg.factory.Nd4j;
import org.nd4j.linalg.api.ndarray.INDArray;
import org.nd4j.linalg.dataset.SplitTestAndTrain;
import org.nd4j.linalg.dataset.DataSet;
import org.deeplearning4j.optimize.listeners.ScoreIterationListener ;

import java.awt.image.BufferedImage;

import org.datavec.image.loader.NativeImageLoader;
import org.nd4j.linalg.api.ndarray.INDArray;

import javax.swing.*;
import java.awt.*;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;

import org.deeplearning4j.util.ModelSerializer;

import java.util.Date;
import java.text.SimpleDateFormat;

In [82]:
public  static void writeNDArrayToImage(INDArray array, BufferedImage image) {
    for (int i=0; i<array.shape()[2]; i++) {
        for (int j=0; j<array.shape()[3]; j++) {
            int g, r, b;
            g = (int) array.getDouble(0, 0, i, j);
            r = (int) array.getDouble(0, 1, i, j);
            b = (int) array.getDouble(0, 2, i, j);
            Color color = new Color(r, g, b);
            image.setRGB(j, i, color.getRGB());
        }
    }
}

public  static BufferedImage NDArrayToImage(INDArray array) {
    /*assume array is w*h*c*/
    int W = array.shape()[0];
    int H = array.shape()[1];
    int C = array.shape()[2];
    array.muli(array.gt(0));
    array = array.mul(array.lte(255)).add(array.gt(255).mul(255));
    if (C == 1) {
        INDArray zeros = Nd4j.zerosLike(array);
        array = Nd4j.concat(2, Nd4j.concat(2, array, array), array);
    }
    BufferedImage img = new BufferedImage(H, W, BufferedImage.TYPE_INT_RGB);
    for (int i=0; i<W; i++) {
        for (int j=0; j<H; j++) {
            int g, r, b;
            g = (int) array.getDouble(i, j, 0);
            r = (int) array.getDouble(i, j, 1);
            b = (int) array.getDouble(i, j, 2);
            Color color = new Color(r, g, b);
            img.setRGB(j, i, color.getRGB());
        }
    }
    return img;
}


In [8]:
private File getMostRecentModel(String beginning) {
    File folder = new File("./");
    File[] listOfFiles = folder.listFiles();
    String mostCurrentFile = "";
    for (int i = 0; i < listOfFiles.length; i++) {
        File currentFile = listOfFiles[i];
        if (currentFile.isFile() && currentFile.getName().startsWith(beginning) 
                                 && currentFile.getName().compareTo(mostCurrentFile) > 0) {
        mostCurrentFile = currentFile.getName();
        } 
    }
    if (mostCurrentFile.compareTo("") == 0) return null;
    return new File(mostCurrentFile);
}

In [5]:
private static MultiLayerNetwork createNN() {
        MultiLayerConfiguration conf = new NeuralNetConfiguration.Builder()
            .seed(12345)
            .weightInit(WeightInit.XAVIER)
            .updater(Updater.ADAGRAD)
            .activation(Activation.RELU)
            .l2(.0001)
            .updater(new Sgd(.05))
            .list()
            .layer(0, new DenseLayer.Builder().nIn(784).nOut(250).build())
            .layer(1, new DenseLayer.Builder().nIn(250).nOut(10).build())
            .layer(2, new DenseLayer.Builder().nIn(10).nOut(250).build())
            .layer(3, new OutputLayer.Builder().nIn(250).nOut(784)
                    .lossFunction(LossFunctions.LossFunction.MSE).build())
            .build();
        MultiLayerNetwork net = new MultiLayerNetwork(conf);
        return net;
    }

In [6]:
Random rnd = new Random(12345);
DataSetIterator dsi = new MnistDataSetIterator(100, 50000, false);
ArrayList<INDArray> featuresTrain = new ArrayList<INDArray>();
ArrayList<INDArray> featuresTest = new ArrayList<INDArray>();
ArrayList<INDArray> labelsTest = new ArrayList<INDArray>();

while (dsi.hasNext()) {
    DataSet ds = dsi.next();
    SplitTestAndTrain split = ds.splitTestAndTrain(80, rnd);
    featuresTrain.add(split.getTrain().getFeatures());
    featuresTest.add(split.getTest().getFeatures());
    labelsTest.add(split.getTest().getLabels().argMax(1));
}

[IJava-executor-0] INFO org.nd4j.linalg.factory.Nd4jBackend - Loaded [JCublasBackend] backend
[IJava-executor-0] INFO org.nd4j.nativeblas.NativeOpsHolder - Number of threads used for NativeOps: 32
[IJava-executor-0] INFO org.nd4j.nativeblas.Nd4jBlas - Number of threads used for BLAS: 0
[IJava-executor-0] INFO org.nd4j.linalg.api.ops.executioner.DefaultOpExecutioner - Backend used: [CUDA]; OS: [Linux]
[IJava-executor-0] INFO org.nd4j.linalg.api.ops.executioner.DefaultOpExecutioner - Cores: [8]; Memory: [4.9GB];
[IJava-executor-0] INFO org.nd4j.linalg.api.ops.executioner.DefaultOpExecutioner - Blas vendor: [CUBLAS]
[IJava-executor-0] INFO org.nd4j.linalg.jcublas.ops.executioner.CudaExecutioner - Device Name: [GeForce GT 640]; CC: [3.0]; Total/free memory: [4225040384]


In [65]:
File f = getMostRecentModel("mardas.model");
MultiLayerNetwork net;
if (f != null) {
    System.out.println("Loading model: " + f);
    net = ModelSerializer.restoreMultiLayerNetwork(f, true);
    net.setListeners(new ScoreIterationListener(30));
    net.init();
} else {
    System.out.println("Creating a new model");
    net = createNN();
}

Loading model: mardas.model-2018-55-13-02-55-01


In [66]:
int numEpochs = 30;

for (int i=0; i<numEpochs; i++) {
    for (INDArray data : featuresTrain) {
        net.fit(data, data);
    }
}

[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 9660 is 0.06976469037351764
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 9690 is 0.07140591251406204
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 9720 is 0.057002746647611666
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 9750 is 0.07190075241213548
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 9780 is 0.06681300507139507
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 9810 is 0.06719703362791041
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 9840 is 0.06788123878811476
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners

[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 11460 is 0.05512815886992588
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 11490 is 0.0616365254087239
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 11520 is 0.07223458332050597
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 11550 is 0.05947325520039466
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 11580 is 0.06766770205205022
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 11610 is 0.06486048393882729
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 11640 is 0.0666843324446714
[IJava-executor-8] INFO org.deeplearning4j.optimize.liste

[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 13260 is 0.061649266053263804
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 13290 is 0.060126184881541776
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 13320 is 0.06226385080361724
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 13350 is 0.06817278632033855
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 13380 is 0.07085313290693993
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 13410 is 0.06187817939421324
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 13440 is 0.06223691351345149
[IJava-executor-8] INFO org.deeplearning4j.optimize.l

[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 15060 is 0.05703634978952891
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 15090 is 0.0556678540721683
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 15120 is 0.06298817244871857
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 15150 is 0.061072214343276865
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 15180 is 0.06547965880719209
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 15210 is 0.060932084356539
[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 15240 is 0.06494800230299688
[IJava-executor-8] INFO org.deeplearning4j.optimize.liste

EvaluationInterruptedException: Evaluator was interrupted while executing: 'for (int i=0; i<numEpochs; i++) {
    for (INDArray data : featuresTrain) {
        net.fit(data, data);
    }
}'

In [67]:
SimpleDateFormat format = new SimpleDateFormat("yyyy-mm-dd-hh-mm-ss");
String fileName = "mardas.model-"+format.format(new Date());
System.out.println("Saving the model as " + fileName);
ModelSerializer.writeModel(net, new File(fileName), true);

Saving the model as mardas.model-2018-03-13-03-03-19


In [68]:
INDArray scores = Nd4j.zeros(20);
for (int i=0; i<20; i++) {
    scores.putScalar(i, net.score(new DataSet(featuresTest.get(0).getRow(i), featuresTest.get(0).getRow(i))));
}

[IJava-executor-8] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 16770 is 0.05560352300705088


In [69]:
scores.mean()

0.0945

In [70]:
net.score(new DataSet(featuresTest.get(0), featuresTest.get(0)))

0.05996525377985682

In [71]:
INDArray diff = net.output(featuresTest.get(0)).sub(featuresTest.get(0))

In [72]:
diff.mul(diff).mean(1)

[0.0608, 
 0.0724, 
 0.0912, 
 0.0678, 
 0.0491, 
 0.0616, 
 0.0747, 
 0.0454, 
 0.0754, 
 0.0308, 
 0.0736, 
 0.0541, 
 0.0376, 
 0.0525, 
 0.0560, 
 0.0553, 
 0.0375, 
 0.1075, 
 0.0375, 
 0.0201]

In [83]:
NDArrayToImage(featuresTest.get(3).getRow(0).reshape(28, 28, 1).mul(255))

In [84]:
NDArrayToImage(net.output(featuresTest.get(3).getRow(0)).reshape(28, 28, 1).mul(255))

28.0